# Modelo preditivo de Classificação para prever o valor de uma variável binária 

In [0]:
# Import dos módulos
import numpy as np
import pandas as pd

from sklearn.decomposition import PCA   
!pip install pca 

from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression

# Não visualizar mais warnings
import warnings
warnings.filterwarnings("ignore")

## Criando Massas de Dados Aleatórias

### Dados de Treino

In [0]:
# Dados de treino
n_train = 10        #variável
np.random.seed(0)

# data frame com 4 variáveis preditoras e 1 variável target
df_treino = pd.DataFrame({"var1": np.random.random(n_train), \
                          "var2": np.random.random(n_train), \
                          "var3": np.random.random(n_train), \
                          "var4": np.random.randint(0,2,n_train).astype(bool),\

                          "target": np.random.randint(0,2,n_train).astype(bool)})


### Dados de Teste

In [0]:
# Dados de teste
n_test = 3
np.random.seed(1)

# data frame com 4 variáveis preditoras e 1 variável target
df_teste = pd.DataFrame({"var1": np.random.random(n_test), \
                         "var2": np.random.random(n_test), \
                         "var3": np.random.random(n_test), \
                         "var4": np.random.randint(0,2,n_test).astype(bool),\
                         "target": np.random.randint(0,2,n_test).astype(bool)})

## Reduzir a dimensionalidade para 3 componentes com PCA( ) 

Reduzir a informação nas 4 variáveis preditoras e **compactar** para 3 
componentes

Componentes armazenarão a informação que está nas 4 variáveis

In [0]:
 # Reduzindo a dimensionalidade para 3 componentes
 pca = PCA(n_components= 3)

In [48]:
type(pca)

sklearn.decomposition._pca.PCA

## Aplicar o **PCA** de dimensionalidade aos conjuntos: **Dados Treino | Dados Teste**

PCA é um algoritmo de aprendizado **Não-Supervisionado**. Na prática não podemos entregar ao algoritmo a variável Target. 

In [0]:
# Aplicar o PCA aos datasets e removemos a variável target com drop() - removendo a variável de saída
newdf_treino = pca.fit_transform(df_treino.drop("target", axis =1))
newdf_teste = pca.transform(df_teste.drop("target", axis = 1))

In [44]:
newdf_treino

array([[ 0.41587301, -0.71062799, -0.11371247],
       [-0.33697959,  0.03003896, -0.17654096],
       [ 0.63605658, -0.29090443,  0.05028171],
       [-0.43846812, -0.20849886,  0.15980423],
       [ 0.94815762,  0.2527858 ,  0.03898095],
       [-0.12006504,  0.42440996, -0.31974987],
       [ 0.0903311 ,  0.76011682,  0.04769318],
       [-0.52214167, -0.27678453, -0.15929367],
       [-0.39311411, -0.02468934,  0.04231957],
       [-0.27964979,  0.04415361,  0.43021735]])

In [45]:
newdf_teste

array([[-0.01816061,  0.5477751 ,  0.20576276],
       [-0.06962209,  0.53031446, -0.12958389],
       [ 0.07690527,  0.63282594,  0.12999804]])

## Converter os dois datasets reduzidos e sem variável target para DataFrame Pandas

Manipulação mais prática na Regressão Logística com formato DataFrame

In [0]:
# Criar dataframes do pandas com o resultado do item anterior
features_treino = pd.DataFrame(newdf_treino)
features_teste = pd.DataFrame(newdf_teste)

In [42]:
features_treino

,0,1,2
0,0.415873,-0.710628,-0.113712
1,-0.336980,0.030039,-0.176541
2,0.636057,-0.290904,0.050282
3,-0.438468,-0.208499,0.159804
4,0.948158,0.252786,0.038981
5,-0.120065,0.424410,-0.319750
6,0.090331,0.760117,0.047693
7,-0.522142,-0.276785,-0.159294
8,-0.393114,-0.024689,0.042320
9,-0.279650,0.044154,0.430217


In [43]:
features_teste

,0,1,2
0,-0.018161,0.547775,0.205763
1,-0.069622,0.530314,-0.129584
2,0.076905,0.632826,0.129998


## Criar Modelo de **Regressão Logística**

In [0]:
# Crie um modelo de regressão logística
regr = LogisticRegression()

##sklearn.pipeline.Pipeline

1. Reduzir Dimensionalidade com PCA
2. Entregar resultado do PCA ao algoritmo de Regressão Logística
3. Criar Pipeline para concatenar  o resultado do PCA e Regressão Logística

In [0]:
# Usando o recurso de pipeline do scikit-learn para encadear 2 algoritmos em um mesmo modelo, 
pipe = Pipeline([("pca", pca),          # Primeiro Par (label, função de Redução)
                 ("logistic", regr)])   # Segundo Par  (label, função de Regressão Linear)
                                        # Alocar no objeto pipe

In [52]:
# Chamar a função fit() do objeto pipe, TREINAR o pipe. De uma única vez treinar PCA e Regressão Logística
pipe.fit(features_treino,     # Primeiro parâmetro: componentes principais, variáveis de entrada
         df_treino["target"]) # Segundo parâmetro: filtrando apenas a variável "target"

Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power='auto', n_components=3,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('logistic',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

## Previsões com o **Modelo Treinado**

In [0]:
# Fazer previsões com o modelo treinado, apenas com dados de entrad
predictions = pipe.predict(features_teste)

In [54]:
# Imprimindo as previsões
print("\Previsões do modelo:")
print(predictions)

\Previsões do modelo:
[ True  True  True]
